# 优化模型参数

现在我们有了模型和数据，是时候通过优化参数训练、验证和测试我们的模型了。
训练模型是一个迭代的过程；在每次迭代中，模型对输出进行预测，计算其预测的误差（损失），收集误差相对于其参数的导数（如我们在上一节所看到的），并使用梯度下降法优化这些参数

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

# 超参数
超参数是可调的参数，允许您控制模型优化过程。不同的超参数值可以影响模型训练和收敛速度（了解更多关于超参数调整的信息）。
我们为训练定义以下超参数：
- Number of Epochs - 对数据集迭代的次数。
- Batch Size - 在更新参数之前，通过网络传播的数据样本数量。
- Learning Rate - 每个批次/轮次更新模型参数的幅度。较小的值会导致学习速度较慢，而较大的值可能在训练过程中导致不可预测的行为。

In [4]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

# 优化循环
一旦我们设置了超参数，我们就可以使用优化循环来训练和优化我们的模型。优化循环的每次迭代称为一个epoch。
每个epoch由两个主要部分组成：
- The Train Loop - 遍历训练数据集，尽量收敛到最佳参数。
- The Validation/Test Loop - 遍历测试数据集，以检查模型性能是否在改善。

让我们简要熟悉训练循环中使用的一些概念。跳到这里查看优化循环的完整实现。

# 损失函数
当给定一些训练数据时，我们的未训练网络可能不会给出正确的答案。

损失函数衡量获得结果与目标值之间的不相似程度，而我们希望在训练过程中最小化的正是损失函数。

为了计算损失，我们使用给定数据样本的输入进行预测，并将其与真实数据标签值进行比较。

常见的损失函数包括用于回归任务的 nn.MSELoss（均方误差）和用于分类的 nn.NLLLoss（负对数似然）。

nn.CrossEntropyLoss 结合了 nn.LogSoftmax 和 nn.NLLLoss。我们将模型的输出 logits 传递给 nn.CrossEntropyLoss，它会对 logits 进行归一化并计算预测误差。

In [5]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

# 优化
优化是调整模型参数以减少每个训练步骤中的模型误差的过程。优化算法定义了如何执行这个过程（在这个例子中我们使用随机梯度下降）。

所有的优化逻辑都封装在优化器对象中。在这里，我们使用SGD优化器；

此外，PyTorch中还有许多不同的优化器，如ADAM和RMSProp，它们对不同类型的模型和数据表现更好。我们通过注册需要训练的模型参数并传入学习率超参数来初始化优化器。

In [6]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

在训练循环中，优化分三个步骤进行：
- 调用 optimizer.zero_grad（） 重置模型参数的梯度。默认情况下，梯度累加;为了防止重复计数，我们在每次迭代时都将它们显式归零。
- 通过调用 loss.backward（） 反向传播预测损失。PyTorch 会根据每个参数来存储损失的梯度。
- 一旦我们有了梯度，我们调用 optimizer.step（） 来根据向后传递中收集的梯度来调整参数。

# 完整实现

我们定义了循环访问优化代码的 train_loop，以及根据测试数据评估模型性能的 test_loop。

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # 计算预测值和损失函数
        pred = model(X)
        loss = loss_fn(pred, y)

        # 反向传播
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # 设置模型为评估模式 - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

我们初始化 loss function 和 optimizer，并将其传递给 train_loop 和 test_loop。

随意增加 epoch 的数量来跟踪模型的改进性能。

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.316443  [   64/60000]
loss: 2.302399  [ 6464/60000]
loss: 2.285570  [12864/60000]
loss: 2.270591  [19264/60000]
loss: 2.250449  [25664/60000]
loss: 2.231316  [32064/60000]
loss: 2.228149  [38464/60000]
loss: 2.199682  [44864/60000]
loss: 2.190567  [51264/60000]
loss: 2.160487  [57664/60000]
Test Error: 
 Accuracy: 50.6%, Avg loss: 2.156297 

Epoch 2
-------------------------------
loss: 2.167261  [   64/60000]
loss: 2.158339  [ 6464/60000]
loss: 2.103681  [12864/60000]
loss: 2.111515  [19264/60000]
loss: 2.060671  [25664/60000]
loss: 2.011058  [32064/60000]
loss: 2.027161  [38464/60000]
loss: 1.959697  [44864/60000]
loss: 1.952793  [51264/60000]
loss: 1.882480  [57664/60000]
Test Error: 
 Accuracy: 59.6%, Avg loss: 1.882403 

Epoch 3
-------------------------------
loss: 1.916326  [   64/60000]
loss: 1.886969  [ 6464/60000]
loss: 1.776004  [12864/60000]
loss: 1.803933  [19264/60000]
loss: 1.695318  [25664/60000]
loss: 1.667745  [32064/600